Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [1]:
import rasterio

NAME = 'Jason Ehlenberger'
COLLABORATORS = 'N/A'

---

# Assignment 2:

1) Create zonal statistics (basically the whole step) for these datasets:   
counties_subset_Albers4.shp   
crop_cover_2019.tif
2) In the zonal statistics, only include these crops: 1, 5, 6, 23, 24, 26, 225,226, 236, 237, 238, 240, 241, 254.
3) From the zonal stat table, delete column "sum".
4) What is the unit of the zonal statistics result? 

What does the crop_2018_subset represent:   
1:Corn, 5:Soybeans, 6:Sunflower, 23:Spring Wheat, 24:Winter Wheat, 26:Dbl Crop WinWht/Soybeans   
225:Dbl Crop WinWht/Corn, 226:Dbl Crop Oats/Corn, 236:Dbl Cropp WinWht/Sorghum, 237:Dbl Crop Barley/Corn,    
238:Dbl Crop WinWht/Cotten, 240:Dbl Crop Soybeans/Oats, 241:bl Crop Corn/Soybeans, 254: Dbl Crop Barley/Soybeans,    
To inlcude: 1, 5, 6, 23, 24, 26, 225,226, 236, 237, 238, 240, 241, 254   

In [2]:
# !conda install -c conda-forge jupyterlab

In [ ]:
  # Use the new function to load data (shapefile in this case)
filename = 'counties_subset_Albers4.shp'
data = load_data(src_dir, filename)

# Load raster data
raster_filename = 'crop_cover_2019.tif'
raster = rasterio.open(load_data(src_dir, raster_filename))

type(raster)

In [ ]:
raster.crs

In [ ]:
# raster.head()

In [ ]:
with rasterio.open(os.path.join(src_dir, 'crop_cover_2019.tif')) as raster_crop:
    crop_val = raster_crop.read()
crop_val

In [ ]:
# Packages 
from rasterio.warp import calculate_default_transform, reproject, Resampling

In [ ]:
raster.bounds

In [ ]:
# Raster resolution 
print(raster.crs.linear_units)

In [ ]:
dst_crs = 'EPSG:9712'

In [ ]:
# step 1: Calculate transform array and shape of reprojected raster 
with rasterio.open(os.path.join(src_dir, 'crop_cover_2019.tif')) as src:
    transform, width, height = calculate_default_transform(
        src.crs, dst_crs, src.width, src.height, *src.bounds)

In [ ]:
    # step 2: We copy metadata from source dataset to dictionary object `kwargs`
    # step 3: we update information in kwargs object with destination crs 
    kwargs = src.meta.copy()
    kwargs.update({
    'crs': dst_crs,
    'transform': transform, 
    'width': width, 
    'height': height
    })

In [ ]:
 # step 4: for each band (i), we reproject the calculated parameters froms source to destination      
    with rasterio.open(os.path.join(src_dir,'crop_2018_subset2_pr.tif', 'w', **kwargs)) as dst:
        for i in range(1, src.count +1):
            reproject(
                source = rasterio.band(src, i), 
                destination = rasterio.band(dst, i), 
                src_transform = src.transform, 
                src_crs = src.crs, 
                dst_transform = transform, 
                dst_crs = dst_crs, 
                resampling = Resampling.nearest)
    # make sure to close the raster file
        dst.close()